<a href="https://colab.research.google.com/github/spirosChv/imbizo2022/blob/main/exercises/exercise5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulating dendrites - Part 5: BAC model

First, we install neuron with python. For more info see [here](https://www.neuron.yale.edu/neuron/download).

In [ ]:
!pip install neuron --quiet

We import some basic python packages and the NEURON package.

## Compile ion channel models (.mod files)

In [ ]:
# @title Create the mod file `SlowCa.mod`
# @markdown Execute this cell.
with open("SlowCa.mod", "w") as file:
  file.write("""
    TITLE HVA Ca2+ current

    COMMENT
    Uses fixed eca instead of GHK eqn
    Based on Reuveni, Friedman, Amitai and Gutnick (1993) J. Neurosci. 13: 4609-4621.
    Changed from (AS Oct0899) ca.mod
    
    Author: Zach Mainen, Salk Institute, 1994, zach@salk.edu
    ENDCOMMENT

    INDEPENDENT {t FROM 0 TO 1 WITH 1 (ms)}

    NEURON {
        SUFFIX sca
        USEION ca READ eca WRITE ica
        RANGE m, h, gca, gbar
        RANGE minf, hinf, mtau, htau, inactF, actF
        GLOBAL q10, temp, tadj, vmin, vmax, vshift
    }

    UNITS {
        (mA) = (milliamp)
        (mV) = (millivolt)
        (pS) = (picosiemens)
        (um) = (micron)
        FARADAY = (faraday) (coulomb)
        R = (k-mole) (joule/degC)
        PI = (pi) (1)
    }

    PARAMETER {
        inactF = 3
        actF = 1
        gbar = 0.1 (pS/um2) : 0.12 mho/cm2
        vshift = 0 (mV) : voltage shift (affects all)

        cao = 2.5 (mM) : external ca concentration
        cai (mM)

        temp = 23 (degC) : original temp 
        q10 = 2.3 : temperature sensitivity

        v (mV)
        dt (ms)
        celsius (degC)
        vmin = -120	(mV)
        vmax = 100 (mV)
    }

    ASSIGNED {
        ica (mA/cm2)
        gca (pS/um2)
        eca (mV)
        minf
        hinf
        mtau (ms)
        htau (ms)
        tadj
    }

    STATE { m h }

    INITIAL { 
        trates(v+vshift)
        m = minf
        h = hinf
    }

    BREAKPOINT {
        SOLVE states
        gca = tadj*gbar*m*m*h
        ica = (1e-4) * gca * (v - eca)
    } 

    LOCAL mexp, hexp

    PROCEDURE states() {
        trates(v+vshift)
        m = m + mexp*(minf-m)
        h = h + hexp*(hinf-h)
        VERBATIM
        return 0;
        ENDVERBATIM
    }

    PROCEDURE trates(v) {
        LOCAL tinc
        TABLE minf, mexp, hinf, hexp
        DEPEND dt, celsius, temp, inactF

        FROM vmin TO vmax WITH 199

        rates(v): not consistently executed from here if usetable == 1

        tadj = q10^((celsius - temp)/10)
        tinc = -dt * tadj

        mexp = 1 - exp(tinc/mtau)
        hexp = 1 - exp(tinc/htau)
    }

    PROCEDURE rates(vm) {  
        LOCAL a, b
        :-27
        :-75
        a = 0.055*(-35 - vm)/(exp((-35-vm)/1) - 1)/actF  :3.8
        b = 0.94*exp((-75-vm)/17)/actF

        mtau = 1/(a+b)
        minf = a*mtau

        :"h" inactivation 

        a = 0.000457*exp((-13-vm)/50)/inactF
        b = 0.0065/(exp((-vm-15)/28) + 1)/inactF

        htau = 1/(a+b) : originally *1
        hinf = a*htau
    }

    FUNCTION efun(z) {
        if (fabs(z) < 1e-4) {
        efun = 1 - z/2
        }else{
        efun = z/(exp(z) - 1)
        }
    }
  """)

In [ ]:
# @title Create the mod file `cad.mod`
# @markdown Execute this cell.
with open("cad.mod", "w") as file:
  file.write("""

    TITLE decay of internal calcium concentration
    
    COMMENT
    : Internal calcium concentration due to calcium currents and pump.
    : Differential equations.
    :
    : Simple model of ATPase pump with 3 kinetic constants (Destexhe 92)
    :     Cai + P <-> CaP -> Cao + P  (k1,k2,k3)
    : A Michaelis-Menten approximation is assumed, which reduces the complexity
    : of the system to 2 parameters: 
    :       kt = <tot enzyme concentration> * k3  -> TIME CONSTANT OF THE PUMP
    :	kd = k2/k1 (dissociation constant)    -> EQUILIBRIUM CALCIUM VALUE
    : The values of these parameters are chosen assuming a high affinity of 
    : the pump to calcium and a low transport capacity (cfr. Blaustein, 
    : TINS, 11: 438, 1988, and references therein).  
    :
    : Units checked using "modlunit" -> factor 10000 needed in ca entry
    :
    : VERSION OF PUMP + DECAY (decay can be viewed as simplified buffering)
    :
    : All variables are range variables
    :
    : adopted from the lower model by AS 102199
    :
    : This mechanism was published in:  Destexhe, A. Babloyantz, A. and 
    : Sejnowski, TJ.  Ionic mechanisms for intrinsic slow oscillations in
    : thalamic relay neurons. Biophys. J. 65: 1538-1552, 1993)
    :
    : Written by Alain Destexhe, Salk Institute, Nov 12, 1992
    :
    ENDCOMMENT

    INDEPENDENT {t FROM 0 TO 1 WITH 1 (ms)}

    NEURON {
        SUFFIX cad
        USEION ca READ ica, cai WRITE cai
        RANGE ca
        GLOBAL depth,cainf,taur
    }

    UNITS {
        (molar) = (1/liter)  : moles do not appear in units
        (mM) = (millimolar)
        (um) = (micron)
        (mA) = (milliamp)
        (msM)	= (ms mM)
        FARADAY = (faraday) (coulomb)
    }

    PARAMETER {
        depth	= .1	(um) : depth of shell
        taur = 100	(ms) : rate of calcium removal, changed from 200 to 80 (H.Markram)
        cainf	= 100e-6 (mM)
        cai (mM)
    }

    STATE {
        ca (mM) 
    }

    INITIAL {
        ca = cainf
    }

    ASSIGNED {
      ica  (mA/cm2)
      drive_channel	(mM/ms)
    }

    BREAKPOINT {
        SOLVE state METHOD euler
    }

    DERIVATIVE state { 
        drive_channel =  - (10000) * ica / (2 * FARADAY * depth)
        if (drive_channel <= 0.) { drive_channel = 0. }	: cannot pump inward

        ca' = drive_channel + (cainf-ca)/taur
        cai = ca
    }
  """)

In [ ]:
# @title Remove old executables and compile with `nrnivmodl`
!rm -rf x86_64/
!nrnivmodl

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from neuron import h
h.load_file("stdrun.hoc")

In [ ]:
# @title Make nicer plots -- Execute this cell
def mystyle():
  """
  Create custom plotting style.

  Returns
  -------
  my_style : dict
      Dictionary with matplotlib parameters.

  """
  # color pallette
  style = {
      # Use LaTeX to write all text
      "text.usetex": False,
      "font.family": "DejaVu Sans",
      "font.weight": "bold",
      # Use 16pt font in plots, to match 16pt font in document
      "axes.labelsize": 16,
      "axes.titlesize": 20,
      "font.size": 16,
      # Make the legend/label fonts a little smaller
      "legend.fontsize": 14,
      "xtick.labelsize": 14,
      "ytick.labelsize": 14,
      "axes.linewidth": 2.5,
      "lines.markersize": 10.0,
      "lines.linewidth": 2.5,
      "xtick.major.width": 2.2,
      "ytick.major.width": 2.2,
      "axes.labelweight": "bold",
      "axes.spines.right": False,
      "axes.spines.top": False
  }

  return style


plt.style.use("seaborn-colorblind")
plt.rcParams.update(mystyle())

Define the simulation parameters

In [ ]:
# Simulation parameters	
tstop = 600  # simulation time (ms)
h.dt = 0.1  # integration step (ms)
vinit = -65  # initial voltage (mV)

### Create the morphology

First, we create the soma.

In [ ]:
soma = h.Section(name='soma')

# Define properties of soma
soma.diam = 20  # diameter (um)
soma.L = 20  # length (um)
soma.Ra = 100  # Axial resistance (Ohm * cm)
soma.cm = 1  # specific membrane capacitance (uF/cm2)
soma.nseg = 1  # number of segments

# Insert the hh channels (inluding leak)
soma.insert('hh')
for seg in soma: 
  seg.hh.gnabar = 0.12  # Sodium conductance (S/cm2)
  seg.hh.gkbar = 0.025  # Potassium conductance (S/cm2)
  seg.hh.gl = 0.00025  # Leak conductance (S/cm2)
  seg.hh.el = -65  # Reversal potential (mV)

## Define the sections!

Then, we create the dendritic tree.

In [ ]:
# Apical trunk sections
trunk0 = h.Section(name='trunk0')
trunk1 = h.Section(name='trunk1')
trunk2 = h.Section(name='trunk2')
trunk3 = h.Section(name='trunk3')
trunk4 = h.Section(name='trunk4')
# Create h.SectionList for easier handling
trunk = h.SectionList()
trunk.append(trunk0)
trunk.append(trunk1)
trunk.append(trunk2)
trunk.append(trunk3)
trunk.append(trunk4)

# Apical tuft sections
tuft0 = h.Section(name='tuft0')
tuft1 = h.Section(name='tuft1')
# Create h.SectionList for easier handling
tuft = h.SectionList()
tuft.append(tuft0)
tuft.append(tuft1)

################################################################################
# Define properties of trunk Sections
diams = [3, 2.5, 2, 1.5, 1.2]  # reducing diameters as we are distal from the soma
for i, sec in enumerate(trunk):
  sec.diam = diams[i]  # diameter (um)
  sec.L = 120 if i < 4 else 20  # length (um)
  sec.Ra = 100  # Axial resistance (Ohm * cm)
  sec.cm = 1  # specific membrane capacitance (uF/cm2)
  sec.nseg = 11  # number of segments

  sec.insert('pas')
  for seg in sec:
    seg.pas.e = -65  # leak reversal potential (mV)
    seg.pas.g = 0.00025  # leak maximal conductance (S/cm2)
################################################################################


################################################################################
# Define properties of trunk Sections
for i, sec in enumerate(tuft):
  sec.diam = 1.0  # diameter (um)
  sec.L = 100  # length (um)
  sec.Ra = 100  # Axial resistance (Ohm * cm)
  sec.cm = 1  # specific membrane capacitance (uF/cm2)
  sec.nseg = 11  # number of segments

  sec.insert('pas')
  for seg in sec:
    seg.pas.e = -65  # leak reversal potential (mV)
    seg.pas.g = 0.00025  # leak maximal conductance (S/cm2)
################################################################################

In [ ]:
# Insert Calcium channels
trunk4.insert('cad')
trunk4.insert('sca')

for seg in trunk4:
  seg.sca.gbar = 5000  # Ca2+ maximal conductance (S/cm2)

Now, let's connect all compartments.

In [ ]:
trunk0.connect(soma(0.5), 0)  # trunkl0 to `middle` of the soma, `soma(0.5)`
trunk1.connect(trunk0(1), 0)  # first point of trunk1 to last of trunk0
trunk2.connect(trunk1(1), 0)  # first point of trunk2 to last of trunk1
trunk3.connect(trunk2(1), 0)  # first point of trunk3 to last of trunk2
trunk4.connect(trunk3(1), 0)  # first point of trunk4 to last of trunk3

# tuft0 and tuft1 are child branches, they have a common parent trunk4
tuft0.connect(trunk4(1), 0)  # first point of tuft0 to last of trunk4
tuft1.connect(trunk4(1), 0)  # first point of tuft1 to last of trunk4

## Synaptic Stimulation

In [ ]:
# Create a current Clamp starting at 200 ms and with duration 5ms. Amplitude is an argument.     
ic = h.IClamp(soma(0.5))
ic.delay = 200  # ms
ic.dur = 5  # ms
ic.amp = 0.0  # nA

In [ ]:
# Include an EPSP
syn = h.Exp2Syn(tuft0(0.5))
syn.tau1 = 0.1  # rise time
syn.tau2 = 20  # decay time
syn.e = 0  # reversal potential of the synapse

#========== ...create an artificial spike (an "event" to be delivered to the synapse)...
ns = h.NetStim(0.5)
ns.start = 200  # stimulus onset
ns.number = 1  # number of events

#... and connect the event to the synapse.
nc = h.NetCon(ns, syn)
nc.delay = 0  # synaptic delay (ms)
nc.weight[0] = 0.005  # synaptic weight (strength)

In [ ]:
vsoma_vec = h.Vector().record(soma(0.5)._ref_v)  # Membrane potential vector
vtrunk4_vec = h.Vector().record(trunk4(0.5)._ref_v)  # Membrane potential vector
t_vec = h.Vector().record(h._ref_t)  # Time stamp vector

# Run the simulation
h.finitialize(vinit)
h.continuerun(tstop)

# Remove the first 20ms to avoid artifacts
tremove = 20
vsoma_vec.remove(0, int(tremove/h.dt))
vtrunk4_vec.remove(0, int(tremove/h.dt))
t_vec.remove(0, int(tremove/h.dt))

plt.figure(figsize=(8, 6))
plt.plot(t_vec, vsoma_vec, label='soma')
plt.plot(t_vec, vtrunk4_vec, label='trunk4')
plt.xlabel('time (ms)')
plt.ylabel('mV')
plt.legend()
plt.show()

### Task: Stimuli combinations

Try different combinations:

1. Turn off both inputs
2. Turn off somatic input, turn on dendritic one.
3. Turn off dendritic input, turn on somatic input
4. Turn on both inputs!

Try to reproduce bursting somatic firing!

In [ ]:
# Stimuli combinations
current_soma = 0.0  # @param {type:"number"}
weight_syn = 0.005 # @param {type:"number"}
ic.amp = current_soma
nc.weight[0] = weight_syn
ic.delay = 200  # ms

# Run the simulation
h.finitialize(vinit)
h.continuerun(tstop)

# Remove the first 20ms to avoid artifacts
tremove = 20
vsoma_vec.remove(0, int(tremove/h.dt))
vtrunk4_vec.remove(0, int(tremove/h.dt))
t_vec.remove(0, int(tremove/h.dt))

plt.figure(figsize=(8, 6))
plt.plot(t_vec, vsoma_vec, label='soma')
plt.plot(t_vec, vtrunk4_vec, label='trunk4')
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')
plt.legend()
plt.show()